上一节所构造的线性神经网络，各计算单元（模型，损失，优化方法）都是基于原理直接编写的。但面对更加复杂的网络时，这样从底层编写显然缺乏效率。利用torch的高级API可以大大加快编写的速度。因此第二节内容中，我们使用torch自带的API编写线性神经网络，并用于之前从底层白那些的神经网络中，各模块的实现进行对比。

In [ ]:
import sys
sys.path.append('..')
import torch as pt
import numpy
from tqdm import tqdm
import torch.utils.data as Data
from torch.nn import init
from linear_regression import *
import torch.optim as optim

首先定义模型。在torch的框架中，已经定义了大部分常用的神经网络模型模板，在实际使用中，我们可以“继承”torch的模板，在此基础上编写我们需要的神经网络结构。继承的模板中必须要包含初始化模型参数（__init__()），以及模板的输出（forward()）函数。

In [ ]:
class linear_network(pt.nn.Module):
    def __init__(self,data_dim):
        super(linear_network,self).__init__()
        self.linear = pt.nn.Linear(data_dim,1)

    def forward(self,x):
        y = self.linear(x)
        return y

在torch的框架下，虽然引入了大量的API加快我们构建网络的速度，但诸如模型参数初始化，模型训练等内容还需要单独定义函数。相对于从底层编写，torch自带的API能简化我们的代码。

In [ ]:
class linear_regression_torch(linear_regression):
    def __init__(self,batch_size=None,learning_rate=None,epoch=None,data_dim=None):
        self.batch_size = batch_size
        self.lr = learning_rate
        self.epoch = epoch
        self.net=linear_network(data_dim=data_dim)

    def generate_train_data(self,x,y):
        dataset = Data.TensorDataset(x,y)
        data_iter = Data.DataLoader(dataset,self.batch_size,shuffle=True)
        return data_iter

    def model_init(self):
        init.normal_(self.net.linear.weight,mean=1,std=0.01)
        init.constant_(self.net.linear.bias,val=0)
        self.loss_function=pt.nn.MSELoss()
        self.optimizer=optim.SGD(self.net.parameters(),lr=self.lr)

    def train_linear_model(self,x,y):
        for i in tqdm(range(self.epoch)):
            for X,Y in self.generate_train_data(x,y):
                loss=self.loss_function(self.net(X),Y.view(-1,1))
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

        print('loss=',loss.item())

In [ ]:
if __name__ == '__main__':
    LinearNet=linear_regression_torch(batch_size=10,learning_rate=0.001,epoch=200,data_dim=3)
    x,y=LinearNet.generate_test_data(3)
    LinearNet.model_init()
    LinearNet.train_linear_model(x,y)
    print(LinearNet.net.linear.weight)